In [5]:
import pandas as pd
from sqlalchemy import *
from create_ref_mariadb import mariadb_connection
from create_ref_mongodb import mongodb_connection
import pymongo
from pprint import pprint
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from from_db_to_df import *

In [16]:
client = mongodb_connection()
eng = mariadb_connection()

df_pc = power_curve(eng)
df_forecast = last_forecast_weather_to_df(client)
df = df_forecast.merge(df_pc, how= "left", on="windfarm_id").round({"wind_speed": 1}).query("wind_speed == windspeed").rename({"power":"power_k"}).reset_index()

df.head()

,index,forecast_date,windfarm_id,temp,feels_like,temp_min,temp_max,pressure,humidity,temp_kf,clouds,wind_speed,wind_deg,wind_gust,windturbine_id,windspeed,power,latitude,longitude
0,55,2023-06-13 09:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,22.31,21.8,22.31,23.4,1011,46,-1.09,38,5.5,76,7.74,1ec6d7ce-93f7-6ef8-abda-53533bbf8f7e,5.5,352.0,48.7136,4.44707
1,306,2023-06-13 09:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,22.31,21.8,22.31,23.4,1011,46,-1.09,38,5.5,76,7.74,1ec6d7ce-93fa-6edc-a613-53533bbf8f7e,5.5,352.0,48.7169,4.45140
2,557,2023-06-13 09:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,22.31,21.8,22.31,23.4,1011,46,-1.09,38,5.5,76,7.74,1ec6d7ce-93fd-6272-a3af-53533bbf8f7e,5.5,352.0,48.7193,4.45693
3,808,2023-06-13 09:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,22.31,21.8,22.31,23.4,1011,46,-1.09,38,5.5,76,7.74,1ec6d7ce-93ff-639c-9afd-53533bbf8f7e,5.5,352.0,48.7212,4.46288
4,1059,2023-06-13 09:00:00,1ec6d7a1-4b96-67a4-9358-df2d2b033685,22.31,21.8,22.31,23.4,1011,46,-1.09,38,5.5,76,7.74,1ec6d7ce-9401-646c-b141-53533bbf8f7e,5.5,352.0,48.7225,4.46990


In [ ]:
def forecast_power_by_turbine(df_forecast_weather: pd.DataFrame, df_power_curve: pd.DataFrame) -> pd.DataFrame:

    # Get estimated power for the 5-next days based on forecast wind and powercurve
    df_final = pd.DataFrame()
    for index, row in df_forecast_weather.iterrows():
        df = df_power_curve[(df_power_curve["windfarm_id"] == row["windfarm_id"]) & (df_power_curve["windspeed"] == round(row["wind_speed"], 1))]
        df_copy = df.copy()
        df_copy.loc[:,"forecast_date"] = row["forecast_date"]
        df_final = pd.concat([df_final, df_copy])
    df_final.rename(columns = {"power": "power_kw"}, inplace = True)
    df_final.drop(columns = ["latitude", "longitude"], inplace=True)
    df_final.sort_values(by=["forecast_date", "windfarm_id", "windturbine_id"], inplace=True)
    df_final.reset_index(inplace=True, drop=True)
    return df_final